The global schema for our application looks like- 
Customer (Name, email, phone#, loyaltyPts, Age Group, medical condition (optional))
Flight (flight#, model, economy seats,business seats, first class seats, price-eco,price-business,price- first class availability, status, departure_time, landing_time, source, destination, distance, delay)

We will be performing schema matching to identify the fields that match with our global schema. We wil be doing this through linguistic matching primarily


Defining wrappers

Importing schemas from databases

In [1]:
import psycopg2

In [ ]:

schemas={}


